In [1]:
import json
from collections import namedtuple
import re

import pandas as pd

Plan = namedtuple("Plan", "context actions")

goldpath_file = "../data/goldsequences-0-1-2-3-4-6-7-8-9-10-11-12-13-14-15-16-17-18-19-20-21-22-23-24-25-26-27-28-29.json"
#goldpath_file = "../data/goldsequences-0.json"

with open(goldpath_file) as file:
    data = json.load(file)


In [2]:
len(data['0']['goldActionSequences'])

30

open/close OBJ - open/close a container
de/activate OBJ - activate/deactivate a device
connect OBJ to OBJ - connect electrical components
disconnect OBJ - disconnect electrical components
use OBJ [on OBJ] - use a device/item
look around describe - the current room
look at OBJ - describe an object in detail
look in OBJ - describe a container’s contents
read OBJ - read a note or book
move OBJ to OBJ - move an object to a container
pick up OBJ - move an object to the inventory
put down OBJ - drop an inventory item
pour OBJ into OBJ - pour a liquid into a container
dunk OBJ into OBJ - dunk a container into a liquid
mix OBJ - chemically mix a container
go to LOC - move to a new location
eat OBJ - eat a food
flush OBJ - flush a toilet
focus on OBJ - signal intent on a task object

In [3]:
def parse(observation: str, inventory: str = "") -> list[str]:
    try:
        x = re.search(r"([\S\s]*?)(?:In it, you see:)([\S\s]*?)(?:You also see:)([\S\s]*)", observation)
        if x == None:
            x = re.search(r"([\S\s]*?)(?:Here you see:)([\S\s]*?)(?:You also see:)([\S\s]*)", observation)
        groups = x.groups()

        location = groups[0]
        objects = groups[1]
        doors = groups[2]

        loc_split = [location.strip()]
        obs_split = [obs.strip() for obs in objects.split('\n') if len(obs.strip()) > 0]
        obs_split = [f"You see {obs}" for obs in obs_split]
        doors_split = [door.strip() for door in doors.split('\n') if len(door.strip()) > 0]
        inventory_items = inventory.replace('\n', ' ').replace('\t', '')
        return loc_split + obs_split + doors_split + [inventory_items]
    except Exception:
        print(observation)

In [4]:
#action = "open door to bathroom"
action = "move metal pot to sink"

matcher = re.search(r"(?:move)([\S\s]*?)to([\S\s]*)", action)
if matcher:
    print([g.strip() for g in matcher.groups()])

['metal pot', 'sink']


dict_keys(['0', '1', '2', '3', '4', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29'])

In [32]:
variation = 3
all_plans = {}
for task_id in list(data.keys())[:4]:
    for variation in range(0, len(data[task_id]['goldActionSequences'])):
        trajectory = data[task_id]['goldActionSequences'][variation]
        task = trajectory['taskDescription']
        goal = task.split('.')[0]
        if goal not in all_plans:
            all_plans[task_id+'_'+goal] = []

        plan = []
        previous_state = parse(trajectory['path'][0]['freelook'])
        for step in trajectory['path'][1:]:
            current_state = parse(step['freelook'])
            action = step['action']
            preconditions = list(set(previous_state) - set(current_state))
            #plan.append((action, step['observation']))
            if action != 'look around':
                plan.append(action)
            previous_state = current_state
            if step['isCompleted'] == 'true':
                break

        all_plans[task_id+'_'+goal].append(plan)

print("ok")


ok


In [33]:
all_plans.keys()

dict_keys(['0_Your task is to boil water', '0_Your task is to boil ice', '0_Your task is to boil orange juice', '0_Your task is to boil apple juice', '0_Your task is to boil chocolate', '0_Your task is to boil marshmallow', '0_Your task is to boil ice cream', '0_Your task is to boil soap', '0_Your task is to boil rubber', '0_Your task is to boil lead', '0_Your task is to boil tin', '0_Your task is to boil mercury', '0_Your task is to boil gallium', '0_Your task is to boil caesium', '1_Your task is to change the state of matter of water', '1_Your task is to change the state of matter of ice', '1_Your task is to change the state of matter of orange juice', '1_Your task is to change the state of matter of apple juice', '1_Your task is to change the state of matter of chocolate', '1_Your task is to change the state of matter of marshmallow', '1_Your task is to change the state of matter of ice cream', '1_Your task is to change the state of matter of soap', '1_Your task is to change the sta

In [7]:
def common_member(a, b):
    a_set = set(a)
    b_set = set(b)

    # check length
    if len(a_set.intersection(b_set)) > 0:
        return(a_set.intersection(b_set))
    else:
        return("no common elements")

common_plans = {}
for goal in all_plans.keys():
    plans = all_plans[goal]
    common_actions = []

    plan_a = plans[0]

    for plan in plans[1:]:
        common_actions = common_member(plan_a, plan)
        plan_a = common_actions

    common_plans[goal] = common_actions

common_actions

{'activate blast furnace',
 'activate stove',
 'examine liquid caesium',
 'focus on caesium',
 'go to bathroom',
 'go to foundry',
 'go to green house',
 'go to hallway',
 'go to kitchen',
 'go to outside',
 'go to workshop',
 'move caesium to metal pot',
 'move metal pot to blast furnace',
 'move metal pot to stove',
 'open blast furnace',
 'open cupboard',
 'open door to bathroom',
 'open door to foundry',
 'open door to green house',
 'open door to hallway',
 'open door to kitchen',
 'open door to outside',
 'open door to workshop',
 'open drawer in counter',
 'open drawer in cupboard',
 'open freezer',
 'open fridge',
 'open glass jar',
 'open oven',
 'pick up caesium',
 'pick up metal pot',
 'pick up thermometer',
 'use thermometer in inventory on liquid caesium'}

In [8]:
len(all_plans['Your task is to boil gallium'])

2

In [11]:
rows = []


plan_id = 0
for i, (goal, plans) in enumerate(all_plans.items()):
    landmarks = common_plans[goal]
    for plan in plans:
        for k, action in enumerate(plan):
            flag = True if action in landmarks else False
            rows.append({
                'action': action,
                'landmark': flag,
                'plan_id': plan_id,
                'order': k,
                'goal': goal
            })
        plan_id += 1

df = pd.DataFrame(rows).sort_values(['plan_id', 'order'])

In [12]:
df.to_csv("goldpaths.csv", index=False)

In [17]:
import os
for task in all_plans.keys():

    filename = task.replace(" ", "_")
    filepath = f"plans_csv/{filename.lower()}"
    if not os.path.exists(filepath):
        os.makedirs(filepath)

    plans_df = df[df['goal'] == task].sort_values(['plan_id', 'order'])
    for plan_id in plans_df['plan_id'].unique():
        plans_df[plans_df['plan_id'] == plan_id].to_csv(f"{filepath}/{plan_id}.csv", index=False)

In [56]:
for goal in all_plans.keys():
    print(goal)

Your task is to boil water
Your task is to boil ice
Your task is to boil orange juice
Your task is to boil apple juice
Your task is to boil chocolate
Your task is to boil marshmallow
Your task is to boil ice cream
Your task is to boil soap
Your task is to boil rubber
Your task is to boil lead
Your task is to boil tin
Your task is to boil mercury
Your task is to boil gallium
Your task is to boil caesium


In [39]:
common_plans['Your task is to boil water']

{'activate sink',
 'activate stove',
 'deactivate sink',
 'examine substance in metal pot',
 'focus on substance in metal pot',
 'go to kitchen',
 'move metal pot to sink',
 'move metal pot to stove',
 'open cupboard',
 'open door to kitchen',
 'pick up metal pot',
 'pick up thermometer',
 'pour metal pot into metal pot',
 'use thermometer in inventory on substance in metal pot'}

In [40]:
df[df['plan_id'] == 0].sort_values(["goal", "order"])

,action,landmark,plan_id,order,goal
0,open door to kitchen,True,0,0,Your task is to boil water
1,go to kitchen,True,0,1,Your task is to boil water
2,pick up thermometer,True,0,2,Your task is to boil water
3,open cupboard,True,0,3,Your task is to boil water
4,pick up metal pot,True,0,4,Your task is to boil water
5,move metal pot to sink,True,0,5,Your task is to boil water
6,activate sink,True,0,6,Your task is to boil water
7,deactivate sink,True,0,7,Your task is to boil water
8,pick up metal pot,True,0,8,Your task is to boil water
9,focus on substance in metal pot,True,0,9,Your task is to boil water


In [58]:
df[df['plan_id'] == 2].sort_values(["goal", "order"]) # fazer planos NA MÃO

,action,landmark,plan_id,order,goal
61,pick up thermometer,True,2,0,Your task is to boil water
62,open cupboard,True,2,1,Your task is to boil water
63,pick up metal pot,True,2,2,Your task is to boil water
64,move metal pot to sink,True,2,3,Your task is to boil water
65,activate sink,True,2,4,Your task is to boil water
66,deactivate sink,True,2,5,Your task is to boil water
67,pick up metal pot,True,2,6,Your task is to boil water
68,move metal pot to sink,True,2,7,Your task is to boil water
69,activate sink,True,2,8,Your task is to boil water
70,deactivate sink,True,2,9,Your task is to boil water


In [ ]:
for i, row in df.iterrows():
    if not row['landmark'] or i == 0:
        #

In [88]:
obs = """
    This outside location is called the outside. Here see:
	the ground
	an axe
	a substance called wood
	a fountain (containing a substance called water)
	a fire pit (containing nothing)
	a substance called air
	the agent
You also see:
	A door to the kitchen (that is open)
	A door to the foundry (that is open)
	A door to the green house (that is open)
"""

x = re.search(r"([\S\s]*?)(?:Here you see:)([\S\s]*?)(?:You also see:)([\S\s]*)", obs)
x.groups()

AttributeError: 'NoneType' object has no attribute 'groups'

In [23]:
import difflib

def matches(list1, list2):
    while True:
        mbs = difflib.SequenceMatcher(None, list1, list2).get_matching_blocks()
        if len(mbs) == 1: break
        for i, j, n in mbs[::-1]:
            if n > 0: yield list1[i: i + n]
            del list1[i: i + n]
            del list2[j: j + n]

variations = [26, 27] # water

var_actions = []

for var in variations:
    actions = []
    for path in data['0']['goldActionSequences'][var]['path']:
        actions.append(path['action'])

    var_actions.append(actions)
print(var_actions[0])
print(var_actions[1])
list(matches(var_actions[0], var_actions[1]))

[['use thermometer in inventory on gaseous gallium', 'wait1'],
 ['pick up metal pot',
  'open door to outside',
  'go to outside',
  'open door to foundry',
  'go to foundry',
  'open blast furnace',
  'move metal pot to blast furnace',
  'activate blast furnace',
  'examine liquid gallium',
  'use thermometer in inventory on liquid gallium',
  'examine liquid gallium',
  'use thermometer in inventory on liquid gallium',
  'examine liquid gallium',
  'use thermometer in inventory on liquid gallium',
  'examine liquid gallium',
  'use thermometer in inventory on liquid gallium',
  'examine liquid gallium',
  'use thermometer in inventory on liquid gallium',
  'examine liquid gallium',
  'use thermometer in inventory on liquid gallium',
  'examine liquid gallium',
  'use thermometer in inventory on liquid gallium',
  'examine liquid gallium',
  'use thermometer in inventory on liquid gallium',
  'examine liquid gallium',
  'use thermometer in inventory on liquid gallium',
  'examine liqu

In [26]:
data['0']['goldActionSequences'][27]['path']

['look around',
 'open door to hallway',
 'go to hallway',
 'open door to kitchen',
 'go to kitchen',
 'look around',
 'pick up thermometer',
 'open cupboard',
 'pick up metal pot',
 'look around',
 'open freezer',
 'open glass jar',
 'open oven',
 'open drawer in cupboard',
 'open fridge',
 'open drawer in counter',
 'open door to bathroom',
 'go to bathroom',
 'look around',
 'open door to kitchen',
 'go to kitchen',
 'look around',
 'open door to outside',
 'go to outside',
 'look around',
 'open door to foundry',
 'go to foundry',
 'look around',
 'open door to outside',
 'go to outside',
 'look around',
 'open door to green house',
 'go to green house',
 'look around',
 'open door to hallway',
 'go to hallway',
 'look around',
 'open door to workshop',
 'go to workshop',
 'look around',
 'pick up gallium',
 'open door to hallway',
 'go to hallway',
 'open door to kitchen',
 'go to kitchen',
 'focus on gallium',
 'move gallium to metal pot',
 'pick up metal pot',
 'move metal pot t

In [71]:
task_example = data['0']['goldActionSequences'][0] # variation zero
print(task_example['taskDescription'])

plan = task_example['path']


Your task is to boil water. For compounds without a boiling point, combusting the substance is also acceptable. First, focus on the substance. Then, take actions that will cause it to change its state of matter.


In [89]:
#print(task_example['taskDescription'].split('.')[0])
#print(plan[0]['freelook'])

goal = task_example['taskDescription'].split('.')[0]
print(f"----------- goal: {goal} -------------")
actions = []
for idx, trajectory in enumerate(plan):
    action = trajectory['action']
    actions.append(action)
    state = trajectory['freelook']
    new_obs = trajectory['observation']
    inventory = trajectory['inventory']
    #if action != 'look around':
    print("--------------------------")
    print(f"Turn {idx}")
    print(f"State: {state}")
    print(f"inventory: {inventory}")
    print(f"Action: {action}")
    print(f"New obs: {new_obs}")


----------- goal: Your task is to boil water -------------
--------------------------
Turn 0
State: This room is called the hallway. In it, you see: 
	a picture
	a substance called air
	the agent
You also see:
	A door to the green house (that is open)
	A door to the living room (that is open)
	A door to the art studio (that is open)
	A door to the kitchen (that is open)
	A door to the bedroom (that is open)
	A door to the workshop (that is open)

inventory: In your inventory, you see:
	an orange

Action: look around
New obs: This room is called the hallway. In it, you see: 
	a picture
	a substance called air
	the agent
You also see:
	A door to the green house (that is open)
	A door to the living room (that is open)
	A door to the art studio (that is open)
	A door to the kitchen (that is open)
	A door to the bedroom (that is open)
	A door to the workshop (that is open)
--------------------------
Turn 1
State: This room is called the hallway. In it, you see: 
	a picture
	a substance call

In [ ]:
"""
! if your task is to boil an water AND you are not in the kitchen
    then
! go to the kitchen (you should see a door to the kitchen)
    + open door to the kitchen
    + go to the kitchen
! pick up metal pot (you should are in the kitchen)
    + open cupboard
    + pick up metal pot
+ pick up thermometer
! fill metal pot with water (must have metal pot in your inventory and you should see a sink)
    + pour metal pot into sink
    + move metal pot to sink
    + activate sink
    + deactivate sink
    + pick up metal pot
! boil water (You should have metal pot in your inventory with water and a thermometer, and you should see a stove)
    + deactivate stove
    + move metal pot to stove
    + activate stove
    + examine substance in metal pot
    + use thermometer in inventory on substance in metal pot
    + examine substance in metal pot
    + use thermometer in inventory on substance in metal pot
    + examine substance in metal pot
    + use thermometer in inventory on substance in metal pot
    + examine substance in metal pot
    + use thermometer in inventory on substance in metal pot
    + examine substance in metal pot
    + use thermometer in inventory on substance in metal pot
"""

In [87]:
# NLI test
"""
This room is called the kitchen. In it, you see: a substance called soapa painting a counter. On the counter is: a bowl (containing a banana, a potato, a red apple, an orange), a drawer.a sink, which is turned off. In the sink is: nothing.a table. On the table is: a glass cup (containing nothing).a chair. On the chair is: nothing.a freezer. The freezer door is closed. a lightera stopwatch, which is deactivated. a fridge. The fridge door is closed. a substance called aira cupboard. The cupboard door is open. In the cupboard is: a tin cup (containing nothing), a ceramic cup (containing nothing), a drawer.a oven, which is turned off. The oven door is closed. You also see:A door to the outside (that is open)A door to the bathroom (that is open)A door to the hallway (that is open) <SEP> you should see a closed door to the kitchen
"""

['look around',
 'open door to kitchen',
 'go to kitchen',
 'look around',
 'pick up thermometer',
 'open cupboard',
 'pick up metal pot',
 'look around',
 'move metal pot to sink',
 'activate sink',
 'deactivate sink',
 'pick up metal pot',
 'focus on substance in metal pot',
 'pour metal pot into metal pot',
 'pick up metal pot',
 'move metal pot to stove',
 'activate stove',
 'examine substance in metal pot',
 'use thermometer in inventory on substance in metal pot',
 'examine substance in metal pot',
 'use thermometer in inventory on substance in metal pot',
 'examine substance in metal pot',
 'use thermometer in inventory on substance in metal pot',
 'examine substance in metal pot',
 'use thermometer in inventory on substance in metal pot',
 'examine substance in metal pot',
 'use thermometer in inventory on substance in metal pot',
 'examine substance in metal pot',
 'use thermometer in inventory on substance in metal pot',
 'examine substance in metal pot',
 'use thermometer in

In [85]:
your goal is to boil water and (plan context) you are in the kitchen and you see an thermometer then
pick up thermometer
open cupboard
pick up metal pot
move metal pot to sink

SyntaxError: invalid syntax (2811270517.py, line 1)

In [81]:
len([a for a in actions if a != 'look around'])

37

In [66]:
if you are in {location} and you see a {obj}
then

18

In [ ]:
# comparar freelook do estado atual com o anterior
# o que mudar deve continuar no plan context
# o que não mudar, da pra remover no plan context, pois a ação não alterou o estado
# o plan context terá apenas